<a href="https://colab.research.google.com/github/FahmidaNourin/CSE445_Machine_Learning/blob/main/Project_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import numpy as np
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/Programming/Sleep_Research/HMC_Sleep_Stages.csv")

In [30]:
df.head()

,Subject,Sleep_Stage,Epoch,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,SN002,N1,6,0.00034,10.11512,17.85756,17.48293,10.24000,0.00047,17.73268,...,0.00112,0.74927,6.36878,2.87220,0.62439,0.000076,32.71805,43.83220,49.95122,49.95122
1,SN002,N1,7,0.00035,10.73951,17.60780,17.10829,9.11610,0.00048,16.85854,...,0.00070,0.99902,6.86829,3.62146,0.74927,0.000072,32.09366,44.33171,49.95122,49.95122
2,SN002,N1,8,0.00035,9.36585,17.60780,16.98341,9.24098,0.00042,17.35805,...,0.00077,1.49854,6.49366,4.12098,0.74927,0.000069,33.09268,43.95707,49.95122,49.95122
3,SN002,N1,116,0.00054,9.74049,15.98439,15.48488,8.36683,0.00051,14.61073,...,0.00082,0.87415,7.49268,3.49659,0.87415,0.000039,28.34732,43.08293,49.95122,49.95122
4,SN002,N1,117,0.00034,10.11512,16.48390,15.98439,12.23805,0.00034,15.11024,...,0.00084,0.87415,6.86829,2.87220,0.62439,0.000032,28.97171,43.20780,49.95122,49.95122


In [31]:
df.shape

(108451, 78)

In [32]:
df = df.loc[:,df.columns != "Subject"]
df = df.loc[:,df.columns != "Epoch"]

In [33]:
df

,Sleep_Stage,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,SpectralEdge_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,N1,0.00034,10.11512,17.85756,17.48293,10.24000,0.00047,17.73268,25.35024,27.47317,...,0.00112,0.74927,6.36878,2.87220,0.62439,0.000076,32.71805,43.83220,49.95122,49.95122
1,N1,0.00035,10.73951,17.60780,17.10829,9.11610,0.00048,16.85854,25.22537,26.97366,...,0.00070,0.99902,6.86829,3.62146,0.74927,0.000072,32.09366,44.33171,49.95122,49.95122
2,N1,0.00035,9.36585,17.60780,16.98341,9.24098,0.00042,17.35805,26.09951,28.47220,...,0.00077,1.49854,6.49366,4.12098,0.74927,0.000069,33.09268,43.95707,49.95122,49.95122
3,N1,0.00054,9.74049,15.98439,15.48488,8.36683,0.00051,14.61073,21.72878,22.47805,...,0.00082,0.87415,7.49268,3.49659,0.87415,0.000039,28.34732,43.08293,49.95122,49.95122
4,N1,0.00034,10.11512,16.48390,15.98439,12.23805,0.00034,15.11024,22.10341,23.47707,...,0.00084,0.87415,6.86829,2.87220,0.62439,0.000032,28.97171,43.20780,49.95122,49.95122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108446,W,0.00042,9.99024,17.10829,16.35902,7.49268,0.00050,16.85854,26.22439,28.34732,...,0.00070,1.12390,8.11707,4.99512,0.49951,0.000098,32.46829,39.83610,47.45366,36.83902
108447,W,0.00033,10.48976,16.60878,16.10927,8.86634,0.00038,15.73463,24.35122,26.09951,...,0.00106,0.99902,6.86829,3.62146,0.74927,0.000053,28.59707,37.46341,44.33171,27.59805
108448,W,0.00064,10.48976,18.73171,18.48195,8.86634,0.00106,19.73073,29.09659,32.46829,...,0.00106,0.74927,8.74146,4.12098,0.37463,0.000320,34.21659,41.33463,50.07610,30.34537
108449,W,0.00051,10.11512,17.85756,17.48293,11.61366,0.00075,19.73073,29.72098,32.96780,...,0.00508,0.87415,5.36976,1.99805,0.99902,0.000240,35.71512,41.83415,50.70049,38.21268


In [34]:
df = df.dropna().reset_index(drop=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108451 entries, 0 to 108450
Data columns (total 76 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Sleep_Stage            108451 non-null  object 
 1   MeanP_Alpha_F4         108451 non-null  float64
 2   MedianF_Alpha_F4       108451 non-null  float64
 3   MeanF_Alpha_F4         108451 non-null  float64
 4   SpectralEdge_Alpha_F4  108451 non-null  float64
 5   PeakF_Alpha_F4         108451 non-null  float64
 6   MeanP_Beta_F4          108451 non-null  float64
 7   MedianF_Beta_F4        108451 non-null  float64
 8   MeanF_Beta_F4          108451 non-null  float64
 9   SpectralEdge_Beta_F4   108451 non-null  float64
 10  PeakF_Beta_F4          108451 non-null  float64
 11  MeanP_Theta_F4         108451 non-null  float64
 12  MedianF_Theta_F4       108451 non-null  float64
 13  MeanF_Theta_F4         108451 non-null  float64
 14  SpectralEdge_Theta_F4  108451 non-nu

In [36]:
target="Sleep_Stage"

In [37]:
df["Sleep_Stage"].value_counts()

N2    39428
N3    21290
W     19355
R     16465
N1    11913
Name: Sleep_Stage, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder
encd = LabelEncoder()

In [39]:
df["Sleep_Stage"] = encd.fit_transform(df["Sleep_Stage"])

In [40]:
X = df.loc[:,df.columns != target]
y = df[target]

In [41]:
df["Sleep_Stage"].value_counts()

1    39428
2    21290
4    19355
3    16465
0    11913
Name: Sleep_Stage, dtype: int64

In [42]:
target="Sleep_Stage"

In [43]:
X = df.loc[:,df.columns != target]
y = df[target]

In [44]:
!pip install imbalanced-learn

In [45]:
y.value_counts()

1    39428
2    21290
4    19355
3    16465
0    11913
Name: Sleep_Stage, dtype: int64

In [46]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_new,y_new = smote.fit_resample(X,y)

In [47]:
y_new.value_counts()

0    39428
1    39428
2    39428
3    39428
4    39428
Name: Sleep_Stage, dtype: int64

In [48]:
from imblearn.under_sampling import RandomUnderSampler

rand = RandomUnderSampler()
X_new2,y_new2 = rand.fit_resample(X,y)

In [49]:
y_new2.value_counts()

0    11913
1    11913
2    11913
3    11913
4    11913
Name: Sleep_Stage, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_new2,y_new2, test_size=0.20)

In [51]:
Y_train.value_counts()

1    9598
3    9555
0    9512
4    9505
2    9482
Name: Sleep_Stage, dtype: int64

In [52]:
Y_test.shape

(11913,)

# tree

In [53]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf=clf.fit(X_train,Y_train)
Y_pred = clf.predict(X_test)

In [54]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))
print("Accurecy: ",accuracy_score(Y_test, Y_pred))

[[1102  355   76  403  465]
 [ 390 1194  399  264   68]
 [  99  398 1858   50   26]
 [ 463  258   56 1481  100]
 [ 565   87   20   86 1650]]
              precision    recall  f1-score   support

           0       0.42      0.46      0.44      2401
           1       0.52      0.52      0.52      2315
           2       0.77      0.76      0.77      2431
           3       0.65      0.63      0.64      2358
           4       0.71      0.69      0.70      2408

    accuracy                           0.61     11913
   macro avg       0.62      0.61      0.61     11913
weighted avg       0.62      0.61      0.61     11913

Accurecy:  0.6115168303533954


#RandomForest

In [55]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

CPU times: user 32.2 s, sys: 107 ms, total: 32.3 s
Wall time: 33.4 s


In [56]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))
print("Accurecy: ",accuracy_score(Y_test, Y_pred))

[[1351  254   18  346  432]
 [ 265 1529  299  165   57]
 [  46  287 2067   10   21]
 [ 212  118   24 1950   54]
 [ 303   18    4   33 2050]]
              precision    recall  f1-score   support

           0       0.62      0.56      0.59      2401
           1       0.69      0.66      0.68      2315
           2       0.86      0.85      0.85      2431
           3       0.78      0.83      0.80      2358
           4       0.78      0.85      0.82      2408

    accuracy                           0.75     11913
   macro avg       0.75      0.75      0.75     11913
weighted avg       0.75      0.75      0.75     11913

Accurecy:  0.7510282884244103


#AdaBoost

In [57]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, Y_train)
Y_pred = ada.predict(X_test)

In [58]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))
print("Accurecy: ",accuracy_score(Y_test, Y_pred))

[[ 755  291   78  552  725]
 [ 224 1059  618  300  114]
 [  54  326 1962   61   28]
 [ 371  312   65 1465  145]
 [ 348   56   41   90 1873]]
              precision    recall  f1-score   support

           0       0.43      0.31      0.36      2401
           1       0.52      0.46      0.49      2315
           2       0.71      0.81      0.76      2431
           3       0.59      0.62      0.61      2358
           4       0.65      0.78      0.71      2408

    accuracy                           0.60     11913
   macro avg       0.58      0.60      0.58     11913
weighted avg       0.58      0.60      0.59     11913

Accurecy:  0.5971627633677495
